# 分词部分

In [1]:
import os 
import re

import jieba
import json
import pandas as pd

In [2]:
### ONLY FOR TEST
def test_segmentation(poem, user_dict=None):   
    for s in poem:
        seg_list = jieba.cut(s, cut_all=False)  
        print('/'.join(list(seg_list)))
        
def run_test_1(user_dict=None):
    poem = ['秦川雄帝宅，函谷壯皇居。', '綺殿千尋起，離宮百雉餘。', '連甍遙接漢，飛觀迥凌虛。', '雲日隱層闕，風煙出綺疎。']
    print('Poem #1')
    test_segmentation(poem, user_dict)
    
    poem = ['遠山媚平楚，宿雨漲清溪。', '沿泝任舟檝，歡言無町畦。', '夜深忽梦少年事，梦啼妆泪红阑干']
    print('Poem #2')
    test_segmentation(poem, user_dict)
    
    poem = ['剑外忽传收蓟北，初闻涕泪满衣裳。',
            '却看妻子愁何在，漫卷诗书喜欲狂。',
            '白日放歌须纵酒，青春作伴好还乡。',
            '即从巴峡穿巫峡，便下襄阳向洛阳。']
    print('Poem #3')
    test_segmentation(poem, user_dict)
    
    poem = ['明月几时有，把酒问青天。', '不知天上宫阙，今夕是何年。', '我欲乘风归去，又恐琼楼玉宇，高处不胜寒。', '起舞弄清影，何似在人间。',
              '转朱阁，低绮户，照无眠。', '不应有恨，何事长向别时圆。', '人有悲欢离合，月有阴晴圆缺，此事古难全。', '但愿人长久，千里共婵娟。']
    print('Poem #4')
    test_segmentation(poem, user_dict)

In [3]:
LYRIC_DIR = './ci/'
POEM_DIR = './json/'

## 唐诗

先测试一下jieba分词的原始分词功能，以下诗句明显分错：
+ 秦川/雄帝/宅/
+ 沿/泝/任舟/檝/，/歡言/無/町/畦/
+ /月/有/阴晴圆/缺/

In [4]:
run_test_1()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.cache


Poem #1


Loading model cost 0.513 seconds.
Prefix dict has been built successfully.


秦川/雄帝/宅/，/函谷壯/皇居/。
綺/殿/千尋起/，/離宮/百雉/餘/。
連/甍/遙/接漢/，/飛觀/迥凌虛/。
雲日/隱層/闕/，/風煙出/綺/疎/。
Poem #2
遠山媚/平楚/，/宿雨漲/清溪/。
沿/泝/任舟/檝/，/歡言/無/町/畦/。
夜深/忽梦/少年/事/，/梦啼妆泪/红/阑干
Poem #3
剑外/忽/传收/蓟北/，/初闻/涕泪/满/衣裳/。
却/看/妻子/愁/何在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还乡/。
即/从/巴峡/穿/巫峡/，/便/下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问青天/。
不知/天上/宫阙/，/今夕/是/何年/。
我/欲/乘风/归去/，/又/恐/琼楼玉宇/，/高处不胜/寒/。
起舞/弄清/影/，/何似/在/人间/。
转朱阁/，/低绮户/，/照无眠/。
不/应有/恨/，/何事/长向/别时圆/。
人有/悲欢离合/，/月/有/阴晴圆/缺/，/此事/古难全/。
但愿人长久/，/千里/共/婵娟/。


接下来我的想法是，先根据五字句的前两个字和七字句的前四个字构建词典，然后以此作为分词词典对整句进行分词。

我用了全唐诗和全宋诗作为数据集来构建词典。经测试，比只使用全唐诗更加精确。

In [5]:
poems = []
file_list = os.listdir(POEM_DIR)
for file in file_list:
    if file.startswith('poet.'):
        poems.append(pd.read_json(f'{POEM_DIR}{file}'))
poem_df = pd.concat(poems, ignore_index=True)

In [6]:
def cut_lines(poem):
    lines = []
    pattern = r',|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）'
    for pairs in poem:
        lines.extend(list(filter(lambda x: len(x) > 0, re.split(pattern, pairs))))
    return lines

In [7]:
poem_df['lines'] = poem_df['paragraphs'].apply(cut_lines)

In [8]:
def first_segment(lines):
    words = []
    for line in lines:
        if len(line) == 5: 
            words.append(line[0:2])
        if len(line) == 7: 
            words.append(line[0:2])
            words.append(line[2:4])
    return words

In [9]:
poem_df['dic'] = poem_df['lines'].apply(first_segment)

In [10]:
word_freq = dict()

dictionary = [item for sublist in poem_df['dic'] for item in sublist]
for i in dictionary:
    word_freq[i] = word_freq.get(i, 0) + 1

word_freq = {key:val for key, val in word_freq.items() if val >= 3}

with open('./poem_dict.txt', 'w', encoding='utf-8') as f:
    for k, v in word_freq.items():
        f.write(f'{k} {v}\n')

jieba.set_dictionary('./poem_dict.txt')

测试目前词典的分词效果。可见之前使用原始词典分错的地方都能分对，但是也添加了几处错误：
+ /梦啼妆泪/红/阑/干
+ /初闻涕/泪满/衣裳/
+ 我欲/乘风/归/去/

但是目前我们只关心唐诗的五字句和七字句的后三个字的分词，所以可以认为效果明显变好，毕竟“雄/帝宅”，“任/舟檝”，“無/町畦”等都能分对。

In [11]:
run_test_1()

Building prefix dict from C:\Users\JUNHONG\history-digital\poem_dict.txt ...


Poem #1


Dumping model to file cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.u8c6200af4b343395bea15d660881757b.cache
Loading model cost 0.266 seconds.
Prefix dict has been built successfully.


秦川/雄/帝宅/，/函谷/壯/皇居/。
綺/殿/千尋/起/，/離宮/百雉/餘/。
連甍/遙接漢/，/飛觀/迥/凌虛/。
雲日/隱層/闕/，/風煙/出/綺疎/。
Poem #2
遠山/媚/平楚/，/宿雨/漲/清溪/。
沿泝/任/舟檝/，/歡言/無/町畦/。
夜深/忽梦/少年/事/，/梦啼妆泪/红/阑/干
Poem #3
剑外/忽/传收/蓟北/，/初闻涕/泪满/衣裳/。
却看/妻子/愁/何在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还/乡/。
即/从/巴峡/穿/巫峡/，/便下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问/青天/。
不知/天上/宫阙/，/今夕/是何/年/。
我欲/乘风/归/去/，/又恐/琼楼/玉宇/，/高处/不胜寒/。
起舞/弄/清影/，/何似/在人/间/。
转朱阁/，/低绮户/，/照无眠/。
不应/有恨/，/何事/长向别/时/圆/。
人有/悲欢/离合/，/月有/阴晴/圆缺/，/此事/古难全/。
但愿/人长/久/，/千里/共婵娟/。


之后根据词典对后三个字再次进行分词，并据此扩充词典：

In [12]:
def addition_segment(phrase):
    result = jieba.lcut(phrase, cut_all=False)     
    if len(result) != 2:
        return phrase
    if len(result[0]) == 2:
        return result[0]
    return result[1]

def second_segment(lines):
    words = []
    for line in lines:
        if len(line) == 5:
            words.append(line[0:2])
            words.append(addition_segment(line[2:]))
        if len(line) == 7:
            words.append(line[0:2])
            words.append(line[2:4])
            words.append(addition_segment(line[4:]))
    return words

In [13]:
poem_df['dic_total'] = poem_df['lines'].apply(second_segment)

In [14]:
word_freq = dict()
dictionary = [item for sublist in poem_df['dic_total'] for item in sublist]
for i in dictionary:
    word_freq[i] = word_freq.get(i, 0) + 1

word_freq = {key:val for key, val in word_freq.items() if val >= 3}

with open('./whole_poem_dict.txt', 'w', encoding='utf-8') as f:
    for k, v in word_freq.items():
        f.write(f'{k} {v}\n')

jieba.set_dictionary('./whole_poem_dict.txt')

第二次的分词效果如下，可见效果再次变好，我觉得和上次的分词效果相当，虽然修正了“初闻/涕泪/满/衣裳”，但是“却看/妻子/愁何/在”又分错了。

In [15]:
run_test_1()

Building prefix dict from C:\Users\JUNHONG\history-digital\whole_poem_dict.txt ...


Poem #1


Dumping model to file cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.u62ee47001558428dfaffbfcc81625e42.cache
Loading model cost 0.277 seconds.
Prefix dict has been built successfully.


秦川/雄/帝宅/，/函谷/壯/皇居/。
綺/殿/千尋/起/，/離宮/百雉/餘/。
連甍/遙接漢/，/飛觀/迥/凌虛/。
雲日/隱層/闕/，/風煙/出/綺疎/。
Poem #2
遠山/媚/平楚/，/宿雨/漲/清溪/。
沿泝/任/舟檝/，/歡言/無/町畦/。
夜深/忽梦/少年/事/，/梦啼妆泪/红/阑/干
Poem #3
剑外/忽/传收/蓟北/，/初闻/涕泪/满/衣裳/。
却看/妻子/愁何/在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还/乡/。
即/从/巴峡/穿/巫峡/，/便下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问/青天/。
不知/天上/宫阙/，/今夕/是何/年/。
我欲/乘风/归/去/，/又恐/琼楼/玉宇/，/高处/不胜寒/。
起舞/弄/清影/，/何似/在人/间/。
转朱阁/，/低绮户/，/照无眠/。
不应/有恨/，/何事/长向别/时/圆/。
人有/悲欢/离合/，/月有/阴晴/圆缺/，/此事/古难全/。
但愿/人长/久/，/千里/共婵娟/。


总结，jieba分词很有问题，词典中单词的频率并不能有效改善分词结果。比如在自定义的词典中，明明没有“愁何”，而且“何在”出现了很多次，但jieba仍然分成了“愁何/在”。

jieba的这个问题已经讨论了好几年了：
https://github.com/fxsjy/jieba/issues/14

## 宋词

我本想人工划分意向，但有些名句出现得太例外：
+ 六字句通常划分方式各异，最常见的为222式，但有的名句与众不同：七八个星天外，两三点雨山前。
+ “满江红”此句与别句的划分方式都不同：“驾长车，踏破贺兰山缺”，加之“贺兰山”是地名，故最好不要人工划分。

所以只能重回之前用的套路，先提取五字句中的前两个字和七字句中的前四字扩充词典，再根据扩充后的词典分词。

In [16]:
lyrics = []
file_list = os.listdir(LYRIC_DIR)
for file in file_list:
    if file.startswith('ci.song'):
        lyrics.append(pd.read_json(f'{LYRIC_DIR}{file}'))
lyric_df = pd.concat(lyrics, ignore_index=True)
lyric_df

,author,paragraphs,rhythmic,tags,prologue
0,和岘,"[气和玉烛，睿化著鸿明。, 缇管一阳生。, 郊禋盛礼燔柴毕，旋轸凤凰城。, 森罗仪卫振华缨。...",导引,NaN,NaN
1,和岘,"[严夜警，铜莲漏迟迟。, 清禁肃，森陛戟，羽卫俨皇闱。, 角声励，钲鼓攸宜。, 金管成雅奏，...",六州,NaN,NaN
2,和岘,"[承宝运，驯致隆平。, 鸿庆被寰瀛。, 时清俗阜，治定功成。, 遐迩咏由庚。, 严郊祀，文物...",十二时・忆少年,NaN,NaN
3,王禹,"[雨恨云愁，江南依旧称佳丽。, 水村渔市。, 一缕孤烟细。, 天际征鸿，遥认行如缀。, 平生...",点绛唇,NaN,NaN
4,苏易简,"[神仙神仙瑶池宴。, 片片。, 碧桃零落春风晚。, 翠云开处，隐隐金舆挽。, 玉麟背冷清风远。]",越江吟,NaN,NaN
...,...,...,...,...,...
21048,范成大,"[催下珠帘护绮丛。, 花枝红里烛枝红。, 烛光花影夜葱茏。, 锦地绣天香雾里，珠星璧月彩云中...",浣溪沙,NaN,NaN
21049,范成大,"[送尽残春更出游。, 风前踪迹似沙鸥。, 浅斟低唱小淹留。, 月见西楼清夜醉，雨添南浦绿波愁...",浣溪沙,NaN,NaN
21050,范成大,"[歙浦钱塘一水通。, 闲云如幕碧重重。, 吴山应在碧云东。, 无力海棠风淡荡，半眠官柳日葱茏...",浣溪沙,NaN,NaN
21051,范成大,"[宝髻双双出绮丛。, 妆光梅影各春风。, 收灯时候却相逢。, 鱼子笺中此宛转，龙香拨上语玲珑...",浣溪沙,NaN,NaN


词牌名概览。出现次数最多的15个词牌名占了宋词的三分之一：

In [17]:
rhythm_df = lyric_df.groupby(['rhythmic']).count()
rhythm_df.sort_values(by='paragraphs', ascending=False).head(15)

,author,paragraphs,tags,prologue
rhythmic,,,,
浣溪沙,776,776,16,1
水调歌头,743,743,45,0
鹧鸪天,657,657,19,0
菩萨蛮,598,598,9,0
满江红,549,549,30,0
西江月,490,490,7,2
临江仙,482,482,13,0
念奴娇,432,432,28,0
减字木兰花,425,425,4,0


In [18]:
lyric_df['lines'] = lyric_df['paragraphs'].apply(cut_lines)

In [19]:
lyric_df['dic'] = lyric_df['lines'].apply(first_segment)
lyric_df['dic']

0        [睿化, 缇管, 郊禋, 盛礼, 旋轸, 森罗, 仪卫, 载路, 皇图, 大业, 垂象, 九...
1                         [铜莲, 羽卫, 金管, 属景, 盛德, 三秀, 合尧, 蹈咏]
2        [鸿庆, 遐迩, 星拱, 布羽, 明德, 神降, 万乘, 肃銮, 玉帛, 皇泽, 普率, 睿...
3                                     [江南, 依旧, 一缕, 遥认, 谁会]
4                             [神仙, 神仙, 碧桃, 零落, 隐隐, 玉麟, 背冷]
                               ...                        
21048     [催下, 珠帘, 花枝, 红里, 烛光, 花影, 锦地, 绣天, 珠星, 璧月, 人间, 别有]
21049     [送尽, 残春, 风前, 踪迹, 浅斟, 低唱, 月见, 西楼, 雨添, 南浦, 有人, 无计]
21050     [歙浦, 钱塘, 闲云, 如幕, 吴山, 应在, 无力, 海棠, 半眠, 官柳, 眼前, 春色]
21051     [宝髻, 双双, 妆光, 梅影, 收灯, 时候, 鱼子, 笺中, 龙香, 拨上, 明朝, 车马]
21052     [红锦, 障泥, 解鞍, 呼渡, 马骄, 不肯, 茅店, 竹篱, 绛裙, 青袂, 临平, 风物]
Name: dic, Length: 21053, dtype: object

In [20]:
word_freq = dict()
dictionary = [item for sublist in poem_df['dic'] for item in sublist]
lyric_dictionary = [item for sublist in lyric_df['dic'] for item in sublist]
dictionary.extend(lyric_dictionary)
for i in dictionary:
    word_freq[i] = word_freq.get(i, 0) + 1

word_freq = {key:val for key, val in word_freq.items() if val >= 3}
    
with open('./lyric_dict.txt', 'w', encoding='utf-8') as f:
    for k, v in word_freq.items():
        f.write(f'{k} {v * 100}\n')

jieba.set_dictionary('./lyric_dict.txt')

In [21]:
def lyric_second_segment(lines):
    words = []
    for line in lines:
        if len(line) == 5:
            words.append(line[0:2])
            words.append(addition_segment(line[2:]))
        if len(line) == 7:
            words.append(line[0:2])
            words.append(line[2:4])
            words.append(addition_segment(line[4:]))
        else:
            tmp = jieba.lcut(line, cut_all=False)
            words.extend(list(filter(lambda x: len(x) >= 2, tmp)))
    return words

In [22]:
lyric_df['dic_total'] = lyric_df['lines'].apply(lyric_second_segment)

Building prefix dict from C:\Users\JUNHONG\history-digital\lyric_dict.txt ...
Dumping model to file cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.uaf15fae91440152dd276ee8f618272a6.cache
Loading model cost 0.256 seconds.
Prefix dict has been built successfully.


In [23]:
run_test_1()

Poem #1
秦川/雄/帝宅/，/函谷/壯/皇居/。
綺/殿/千尋/起/，/離宮/百雉/餘/。
連甍/遙接漢/，/飛觀/迥/凌虛/。
雲日/隱層/闕/，/風煙/出/綺疎/。
Poem #2
遠山/媚/平楚/，/宿雨/漲/清溪/。
沿泝/任/舟檝/，/歡言/無/町畦/。
夜深/忽梦/少年/事/，/梦/啼妆/泪红/阑干
Poem #3
剑外/忽/传收/蓟北/，/初闻/涕泪/满/衣裳/。
却看/妻子/愁/何在/，/漫卷/诗书/喜欲狂/。
白日/放歌/须/纵酒/，/青春/作伴/好/还/乡/。
即/从/巴峡/穿/巫峡/，/便下/襄阳/向/洛阳/。
Poem #4
明月/几时/有/，/把酒/问/青天/。
不知/天上/宫阙/，/今夕/是何/年/。
我欲/乘风/归去/，/又恐/琼楼/玉宇/，/高处/不胜/寒/。
起舞/弄/清影/，/何似/在人/间/。
转/朱阁/，/低绮户/，/照/无眠/。
不应/有恨/，/何事/长向/别时/圆/。
人有/悲欢/离合/，/月有/阴晴/圆缺/，/此事/古难全/。
但愿/人长/久/，/千里/共/婵娟/。


最后一次分词的结果。“却看/妻子/愁/何在”终于分对了，虽然“梦/啼妆/泪红/阑干”分错，但是我们只关心对后三个字的分词，所以无伤大雅。最关键的是，“转/朱阁”终于分出来了，“千里/共/婵娟”的“婵娟”也分出来了。但是上述几个分词词典都没正确地分出“但愿/人长/久”，我看字典里没有“长久”和“人长”，所以我认为是它随便分了。

总结一下我的想法：

假设：五字句AABBB的前两个字AA，七字句AABBCCC的前四个字AA和BB，统一视为可以直接提取出来的“好意象”。当然这个规律也有例外，但是我只找到白居易的“家在虾蟆陵下住”这一个反例，我觉得只是因为白居易本来就专门写通俗化口语化的诗。后三个字的分法大相径庭，所以需要借助分词工具。

做法：
1. 先用唐诗和宋诗的“好意象”构建词典，然后用得到的词典对五字句AABBB的后三个字BBB，七字句AABBCCC的后三个字CCC分词。用对BBB和CCC的分词结果扩充词典。
2. 唐诗的分词完成。分词结果就是好意象+后三个字的分词。
3. 用宋词里的五字句的和七字句的“好意象”扩充词典，对宋词其他句式进行分词，
4. 宋词的分词完成。分词结果就是：
        对于五字句和七字句：好意象+后三个字的分词；
        对于其他句式：整句的分词结果

In [24]:
song_df = lyric_df

poems = []
file_list = os.listdir(POEM_DIR)
for file in file_list:
    if file.startswith('poet.tang'):
        poems.append(pd.read_json(f'{POEM_DIR}{file}'))
tang_df = pd.concat(poems, ignore_index=True)
tang_df['lines'] = tang_df['paragraphs'].apply(cut_lines)
jieba.set_dictionary('./poem_dict.txt')
tang_df['dic_total'] = tang_df['lines'].apply(second_segment)
jieba.set_dictionary('./lyric_dict.txt')

Building prefix dict from C:\Users\JUNHONG\history-digital\poem_dict.txt ...
Loading model from cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.u8c6200af4b343395bea15d660881757b.cache
Loading model cost 0.163 seconds.
Prefix dict has been built successfully.


总之，目前可以使用tang_df['dic_total']和song_df['dic_total']来看唐诗和宋词的分词结果：

In [25]:
tang_df

,author,paragraphs,title,id,tags,lines,dic_total
0,太宗皇帝,"[秦川雄帝宅，函谷壯皇居。, 綺殿千尋起，離宮百雉餘。, 連甍遙接漢，飛觀迥凌虛。, 雲日隱...",帝京篇十首 一,3ad6d468-7ff1-4a7b-8b24-a27d70d00ed4,NaN,"[秦川雄帝宅, 函谷壯皇居, 綺殿千尋起, 離宮百雉餘, 連甍遙接漢, 飛觀迥凌虛, 雲日隱...","[秦川, 帝宅, 函谷, 皇居, 綺殿, 千尋, 離宮, 百雉, 連甍, 遙接漢, 飛觀, ..."
1,太宗皇帝,"[巖廊罷機務，崇文聊駐輦。, 玉匣啓龍圖，金繩披鳳篆。, 韋編斷仍續，縹帙舒還卷。, 對此乃...",帝京篇十首 二,13e72581-968b-457f-b381-a3b7d95b8b7c,NaN,"[巖廊罷機務, 崇文聊駐輦, 玉匣啓龍圖, 金繩披鳳篆, 韋編斷仍續, 縹帙舒還卷, 對此乃...","[巖廊, 機務, 崇文, 聊駐, 玉匣, 龍圖, 金繩, 鳳篆, 韋編, 仍續, 縹帙, 舒..."
2,太宗皇帝,"[移步出詞林，停輿欣武宴。, 琱弓寫明月，駿馬疑流電。, 驚雁落虛弦，啼猿悲急箭。, 閱賞誠...",帝京篇十首 三,a7ff247d-a11c-4ca9-a22f-ca420b8c537c,NaN,"[移步出詞林, 停輿欣武宴, 琱弓寫明月, 駿馬疑流電, 驚雁落虛弦, 啼猿悲急箭, 閱賞誠...","[移步, 詞林, 停輿, 欣武宴, 琱弓, 明月, 駿馬, 流電, 驚雁, 虛弦, 啼猿, ..."
3,太宗皇帝,"[鳴笳臨樂館，眺聽歡芳節。, 急管韻朱絃，清歌凝白雪。, 彩鳳肅來儀，玄鶴紛成列。, 去茲鄭...",帝京篇十首 四,fa374b2b-c196-4362-b4ad-8931fc9a8860,NaN,"[鳴笳臨樂館, 眺聽歡芳節, 急管韻朱絃, 清歌凝白雪, 彩鳳肅來儀, 玄鶴紛成列, 去茲鄭...","[鳴笳, 臨樂館, 眺聽, 芳節, 急管, 朱絃, 清歌, 凝白, 彩鳳, 來儀, 玄鶴, ..."
4,太宗皇帝,"[芳辰追逸趣，禁苑信多奇。, 橋形通漢上，峰勢接雲危。, 煙霞交隱映，花鳥自參差。, 何如肆...",帝京篇十首 五,86952cb3-b622-4398-a56a-01dd39f6c6ec,NaN,"[芳辰追逸趣, 禁苑信多奇, 橋形通漢上, 峰勢接雲危, 煙霞交隱映, 花鳥自參差, 何如肆...","[芳辰, 追逸趣, 禁苑, 多奇, 橋形, 漢上, 峰勢, 接雲危, 煙霞, 隱映, 花鳥,..."
...,...,...,...,...,...,...,...
57607,梁鍠,"[莫向嵩山去，神仙多誤人。, 不如朝魏闕，天子重賢臣。]",贈李中華,bc3a22df-2cf8-43a7-971b-f0dd9b51e6d6,NaN,"[莫向嵩山去, 神仙多誤人, 不如朝魏闕, 天子重賢臣]","[莫向, 嵩山, 神仙, 多誤, 不如, 魏闕, 天子, 賢臣]"
57608,梁鍠,"[刻木牽絲作老翁，雞皮鶴髮與真同。, 須臾弄罷寂無事，還似人生一夢中。]",詠木老人,8190cf76-f2af-400c-a09d-8877e9b25a62,NaN,"[刻木牽絲作老翁, 雞皮鶴髮與真同, 須臾弄罷寂無事, 還似人生一夢中]","[刻木, 牽絲, 老翁, 雞皮, 鶴髮, 真同, 須臾, 弄罷, 寂無, 還似, 人生, 一夢]"
57609,梁鍠,[堂高憑上望，宅廣乘車行。],句,69b236e5-0a72-4f82-a581-7d8d83122f6c,NaN,"[堂高憑上望, 宅廣乘車行]","[堂高, 憑上望, 宅廣, 乘車]"
57610,杜儼,"[書劒催人不暫閑，洛陽羈旅復秦關。, 容顏歲歲愁邊改，鄉國時時夢裏還。]",客中作,735c5425-745a-4a91-97e3-e6d262dce5fc,NaN,"[書劒催人不暫閑, 洛陽羈旅復秦關, 容顏歲歲愁邊改, 鄉國時時夢裏還]","[書劒, 催人, 暫閑, 洛陽, 羈旅, 秦關, 容顏, 歲歲, 愁邊, 鄉國, 時時, 夢裏]"


In [26]:
song_df

,author,paragraphs,rhythmic,tags,prologue,lines,dic,dic_total
0,和岘,"[气和玉烛，睿化著鸿明。, 缇管一阳生。, 郊禋盛礼燔柴毕，旋轸凤凰城。, 森罗仪卫振华缨。...",导引,NaN,NaN,"[气和玉烛, 睿化著鸿明, 缇管一阳生, 郊禋盛礼燔柴毕, 旋轸凤凰城, 森罗仪卫振华缨, ...","[睿化, 缇管, 郊禋, 盛礼, 旋轸, 森罗, 仪卫, 载路, 皇图, 大业, 垂象, 九...","[和玉, 睿化, 著鸿明, 睿化, 著鸿明, 缇管, 一阳, 一阳, 郊禋, 盛礼, 燔柴,..."
1,和岘,"[严夜警，铜莲漏迟迟。, 清禁肃，森陛戟，羽卫俨皇闱。, 角声励，钲鼓攸宜。, 金管成雅奏，...",六州,NaN,NaN,"[严夜警, 铜莲漏迟迟, 清禁肃, 森陛戟, 羽卫俨皇闱, 角声励, 钲鼓攸宜, 金管成雅奏...","[铜莲, 羽卫, 金管, 属景, 盛德, 三秀, 合尧, 蹈咏]","[夜警, 铜莲, 迟迟, 铜莲漏, 迟迟, 清禁, 陛戟, 羽卫, 俨皇闱, 羽卫, 俨皇闱..."
2,和岘,"[承宝运，驯致隆平。, 鸿庆被寰瀛。, 时清俗阜，治定功成。, 遐迩咏由庚。, 严郊祀，文物...",十二时・忆少年,NaN,NaN,"[承宝运, 驯致隆平, 鸿庆被寰瀛, 时清俗阜, 治定功成, 遐迩咏由庚, 严郊祀, 文物声...","[鸿庆, 遐迩, 星拱, 布羽, 明德, 神降, 万乘, 肃銮, 玉帛, 皇泽, 普率, 睿...","[承宝运, 驯致, 隆平, 鸿庆, 寰瀛, 鸿庆, 寰瀛, 时清, 俗阜, 治定, 功成, ..."
3,王禹,"[雨恨云愁，江南依旧称佳丽。, 水村渔市。, 一缕孤烟细。, 天际征鸿，遥认行如缀。, 平生...",点绛唇,NaN,NaN,"[雨恨云愁, 江南依旧称佳丽, 水村渔市, 一缕孤烟细, 天际征鸿, 遥认行如缀, 平生事,...","[江南, 依旧, 一缕, 遥认, 谁会]","[雨恨, 云愁, 江南, 依旧, 佳丽, 水村, 渔市, 一缕, 孤烟, 一缕, 孤烟, 天..."
4,苏易简,"[神仙神仙瑶池宴。, 片片。, 碧桃零落春风晚。, 翠云开处，隐隐金舆挽。, 玉麟背冷清风远。]",越江吟,NaN,NaN,"[神仙神仙瑶池宴, 片片, 碧桃零落春风晚, 翠云开处, 隐隐金舆挽, 玉麟背冷清风远]","[神仙, 神仙, 碧桃, 零落, 隐隐, 玉麟, 背冷]","[神仙, 神仙, 瑶池, 片片, 碧桃, 零落, 春风, 翠云, 开处, 隐隐, 金舆, 隐..."
...,...,...,...,...,...,...,...,...
21048,范成大,"[催下珠帘护绮丛。, 花枝红里烛枝红。, 烛光花影夜葱茏。, 锦地绣天香雾里，珠星璧月彩云中...",浣溪沙,NaN,NaN,"[催下珠帘护绮丛, 花枝红里烛枝红, 烛光花影夜葱茏, 锦地绣天香雾里, 珠星璧月彩云中, ...","[催下, 珠帘, 花枝, 红里, 烛光, 花影, 锦地, 绣天, 珠星, 璧月, 人间, 别有]","[催下, 珠帘, 护绮丛, 花枝, 红里, 烛枝, 烛光, 花影, 夜葱, 锦地, 绣天, ..."
21049,范成大,"[送尽残春更出游。, 风前踪迹似沙鸥。, 浅斟低唱小淹留。, 月见西楼清夜醉，雨添南浦绿波愁...",浣溪沙,NaN,NaN,"[送尽残春更出游, 风前踪迹似沙鸥, 浅斟低唱小淹留, 月见西楼清夜醉, 雨添南浦绿波愁, ...","[送尽, 残春, 风前, 踪迹, 浅斟, 低唱, 月见, 西楼, 雨添, 南浦, 有人, 无计]","[送尽, 残春, 更出, 风前, 踪迹, 沙鸥, 浅斟, 低唱, 小淹, 月见, 西楼, 清..."
21050,范成大,"[歙浦钱塘一水通。, 闲云如幕碧重重。, 吴山应在碧云东。, 无力海棠风淡荡，半眠官柳日葱茏...",浣溪沙,NaN,NaN,"[歙浦钱塘一水通, 闲云如幕碧重重, 吴山应在碧云东, 无力海棠风淡荡, 半眠官柳日葱茏, ...","[歙浦, 钱塘, 闲云, 如幕, 吴山, 应在, 无力, 海棠, 半眠, 官柳, 眼前, 春色]","[歙浦, 钱塘, 一水, 闲云, 如幕, 重重, 吴山, 应在, 碧云, 无力, 海棠, 淡..."
21051,范成大,"[宝髻双双出绮丛。, 妆光梅影各春风。, 收灯时候却相逢。, 鱼子笺中此宛转，龙香拨上语玲珑...",浣溪沙,NaN,NaN,"[宝髻双双出绮丛, 妆光梅影各春风, 收灯时候却相逢, 鱼子笺中此宛转, 龙香拨上语玲珑, ...","[宝髻, 双双, 妆光, 梅影, 收灯, 时候, 鱼子, 笺中, 龙香, 拨上, 明朝, 车马]","[宝髻, 双双, 出绮丛, 妆光, 梅影, 春风, 收灯, 时候, 相逢, 鱼子, 笺中, ..."


In [27]:
libai = tang_df[tang_df['author']== '李白']
li_seg = libai['dic_total'].tolist()

In [28]:
libai

,author,paragraphs,title,id,tags,lines,dic_total
854,李白,"[三十六離宮，樓臺與天通。, 閣道步行月，美人愁煙空。, 恩疎寵不及，桃李傷春風。, 淫樂意...",鼓吹曲辭 上之回,ee3195f6-eb17-4e87-b714-f7cf07ff03f2,"[忧国忧民, 鼓吹曲辞, 乐府]","[三十六離宮, 樓臺與天通, 閣道步行月, 美人愁煙空, 恩疎寵不及, 桃李傷春風, 淫樂意...","[三十, 離宮, 樓臺, 與天, 閣道, 步行, 美人, 愁煙, 恩疎, 不及, 桃李, 傷..."
857,李白,"[去年戰桑乾源，今年戰蔥河道。, 洗兵條支海上波，放馬天山雪中草。, 萬里長征戰，三軍盡衰老...",鼓吹曲辭 戰城南,c0f1c8ed-bbf8-4ace-ac86-91e5e57d8ece,"[忧国忧民, 战争, 鼓吹曲辞, 乐府, 讽刺]","[去年戰桑乾源, 今年戰蔥河道, 洗兵條支海上波, 放馬天山雪中草, 萬里長征戰, 三軍盡衰...","[洗兵, 條支, 海上, 放馬, 天山, 雪中, 萬里, 長征, 三軍, 衰老, 秦家, 築..."
874,李白,"[君不見黃河之水天上來，奔流到海不復回。, 君不見高堂明鏡悲白髮，朝如青絲暮成雪。, 人生得...",鼓吹曲辭 將進酒,0f7504df-cda2-4fe2-bc63-867ec2e418e7,"[唐诗三百首, 黄河, 咏物, 抒情, 鼓吹曲辞, 乐府, 宴饮, 哲理, 水, 咏物诗]","[君不見黃河之水天上來, 奔流到海不復回, 君不見高堂明鏡悲白髮, 朝如青絲暮成雪, 人生得...","[奔流, 到海, 不復, 朝如, 青絲, 成雪, 人生, 得意, 須盡, 莫使, 金尊, 空..."
877,李白,"[君馬黃，我馬白。, 馬色雖不同，人心本無隔。, 共作遊冶盤，雙行洛陽陌。, 長劒既照曜，高...",鼓吹曲辭 君馬黃,d4c03bc1-0596-4177-88ba-e42a97b0a7d9,"[知己, 鼓吹曲辞, 乐府]","[君馬黃, 我馬白, 馬色雖不同, 人心本無隔, 共作遊冶盤, 雙行洛陽陌, 長劒既照曜, ...","[馬色, 雖不, 人心, 本無, 共作, 遊冶, 雙行, 洛陽, 長劒, 照曜, 高冠, 何..."
885,李白,"[我思仙人，乃在碧海之東隅。, 海寒多天風，白波連山倒蓬壺。, 長鯨噴湧不可涉，撫心茫茫淚如...",鼓吹曲辭 有所思,4f876a76-9628-4cc3-91da-124444b562f1,NaN,"[我思仙人, 乃在碧海之東隅, 海寒多天風, 白波連山倒蓬壺, 長鯨噴湧不可涉, 撫心茫茫淚...","[乃在, 碧海, 之東, 海寒, 天風, 白波, 連山, 蓬壺, 長鯨, 噴湧, 不可, 撫..."
...,...,...,...,...,...,...,...
56544,李白,"[我吟傳舍詠，來訪真人居。, 煙嶺迷高跡，雲林隔太虛。, 窺庭但蕭瑟，倚杖空躊躇。, 應化遼...",題許宣平菴壁,f1aa65f7-c533-48c5-a405-ae46a3a6e137,NaN,"[我吟傳舍詠, 來訪真人居, 煙嶺迷高跡, 雲林隔太虛, 窺庭但蕭瑟, 倚杖空躊躇, 應化遼...","[我吟, 傳舍, 來訪, 真人, 煙嶺, 高跡, 雲林, 太虛, 窺庭, 蕭瑟, 倚杖, 躊..."
56545,李白,"[飯顆山頭逢杜甫，頂戴笠子日卓午。, 借問別來太瘦生，總爲從前作詩苦。]",戲贈杜甫,18d50cb2-d2f6-48bf-b4bf-2226429a2bca,"[喜悦, 友情, 重逢]","[飯顆山頭逢杜甫, 頂戴笠子日卓午, 借問別來太瘦生, 總爲從前作詩苦]","[飯顆, 山頭, 杜甫, 頂戴, 笠子, 卓午, 借問, 別來, 太瘦, 總爲, 從前, 作詩]"
56546,李白,"[茫茫南與北，道直事難諧。, 榆莢錢生樹，楊花玉糝街。, 塵縈遊子面，蝶弄美人釵。, 却憶青...",春感詩,2a0011ec-102c-4d51-b204-d5909a1f63eb,NaN,"[茫茫南與北, 道直事難諧, 榆莢錢生樹, 楊花玉糝街, 塵縈遊子面, 蝶弄美人釵, 却憶青...","[茫茫, 南與, 道直, 事難, 榆莢, 生樹, 楊花, 玉糝, 塵縈, 遊子, 蝶弄, 美..."
56547,李白,"[素面倚欄鉤，嬌聲出外頭。, 若非是織女，何得問牽牛。]",白微時募縣小吏入令臥內嘗驅牛經堂下令妻怒將加詰責白亟以詩謝云,8938db72-4784-4d3e-ba8c-f9fe6174dc58,NaN,"[素面倚欄鉤, 嬌聲出外頭, 若非是織女, 何得問牽牛]","[素面, 倚欄, 嬌聲, 出外, 若非, 織女, 何得, 牽牛]"


In [69]:
len(libai['paragraphs'])

1207

In [29]:
li_seg[0]

['三十',
 '離宮',
 '樓臺',
 '與天',
 '閣道',
 '步行',
 '美人',
 '愁煙',
 '恩疎',
 '不及',
 '桃李',
 '傷春',
 '淫樂',
 '意何極',
 '金輿',
 '回中',
 '萬乘',
 '黃道',
 '千旗',
 '彩虹',
 '前軍',
 '細柳',
 '後騎',
 '甘泉',
 '豈問',
 '渭川',
 '寧邀',
 '野童',
 '秋暮',
 '瑤池',
 '歸來',
 '未窮']

In [30]:
import operator
from functools import reduce

li_list = reduce(operator.add, li_seg)
li_list

['三十',
 '離宮',
 '樓臺',
 '與天',
 '閣道',
 '步行',
 '美人',
 '愁煙',
 '恩疎',
 '不及',
 '桃李',
 '傷春',
 '淫樂',
 '意何極',
 '金輿',
 '回中',
 '萬乘',
 '黃道',
 '千旗',
 '彩虹',
 '前軍',
 '細柳',
 '後騎',
 '甘泉',
 '豈問',
 '渭川',
 '寧邀',
 '野童',
 '秋暮',
 '瑤池',
 '歸來',
 '未窮',
 '洗兵',
 '條支',
 '海上',
 '放馬',
 '天山',
 '雪中',
 '萬里',
 '長征',
 '三軍',
 '衰老',
 '秦家',
 '築城',
 '備胡處',
 '漢家',
 '還有',
 '烽火',
 '烽火',
 '不息',
 '征戰',
 '無已',
 '野戰',
 '格鬬',
 '敗馬',
 '號鳴',
 '向天',
 '烏鳶',
 '人腸',
 '銜飛',
 '上挂',
 '枯樹',
 '士卒',
 '草莽',
 '將軍',
 '空爾',
 '乃知',
 '兵者',
 '凶器',
 '奔流',
 '到海',
 '不復',
 '朝如',
 '青絲',
 '成雪',
 '人生',
 '得意',
 '須盡',
 '莫使',
 '金尊',
 '空對',
 '天生',
 '我材',
 '必有',
 '千金',
 '散盡',
 '還復',
 '烹羊',
 '宰牛',
 '且爲',
 '會須',
 '一飲',
 '三百',
 '與君',
 '一曲',
 '請君',
 '爲我',
 '側耳',
 '鐘鼓',
 '饌玉',
 '不足',
 '但願',
 '長醉',
 '不復',
 '古來',
 '聖賢',
 '寂寞',
 '惟有',
 '飲者',
 '其名',
 '陳王',
 '昔時',
 '平樂',
 '斗酒',
 '十千',
 '恣歡',
 '主人',
 '何爲',
 '言少',
 '徑須',
 '酤取',
 '對君',
 '呼兒',
 '將出',
 '美酒',
 '與爾',
 '同銷',
 '萬古',
 '馬色',
 '雖不',
 '人心',
 '本無',
 '共作',
 '遊冶',
 '雙行',
 '洛陽',
 '長劒',
 '照曜',
 '高冠',
 '何赩赫',
 '各有',
 '千金',
 '俱

In [31]:
from collections import Counter

result_li = Counter(li_list)

In [32]:
result_li

Counter({'三十': 19,
         '離宮': 3,
         '樓臺': 5,
         '與天': 9,
         '閣道': 3,
         '步行': 2,
         '美人': 28,
         '愁煙': 1,
         '恩疎': 4,
         '不及': 13,
         '桃李': 42,
         '傷春': 2,
         '淫樂': 3,
         '意何極': 2,
         '金輿': 3,
         '回中': 3,
         '萬乘': 16,
         '黃道': 2,
         '千旗': 2,
         '彩虹': 5,
         '前軍': 2,
         '細柳': 4,
         '後騎': 2,
         '甘泉': 5,
         '豈問': 2,
         '渭川': 4,
         '寧邀': 2,
         '野童': 2,
         '秋暮': 1,
         '瑤池': 7,
         '歸來': 26,
         '未窮': 5,
         '洗兵': 2,
         '條支': 2,
         '海上': 21,
         '放馬': 2,
         '天山': 8,
         '雪中': 4,
         '萬里': 60,
         '長征': 2,
         '三軍': 11,
         '衰老': 5,
         '秦家': 5,
         '築城': 2,
         '備胡處': 1,
         '漢家': 13,
         '還有': 4,
         '烽火': 10,
         '不息': 4,
         '征戰': 8,
         '無已': 2,
         '野戰': 2,
         '格鬬': 2,
         '敗馬': 2,
         '號鳴': 

In [33]:
result_li.most_common(10)

[('不可', 75),
 ('黃金', 68),
 ('春風', 65),
 ('萬里', 60),
 ('白日', 57),
 ('明月', 52),
 ('青天', 49),
 ('相思', 49),
 ('天地', 47),
 ('青雲', 44)]

In [75]:
def tang_author_topK(author,K):
    author_seg = tang_df[tang_df['author']==author]['dic_total'].tolist()
    if len(author_seg)==0:return []
    author_list = reduce(operator.add, author_seg) 
    result_author = Counter(author_list)
    return  result_author.most_common(K)

In [76]:
tang_author_topK('李白',10)

[('不可', 75),
 ('黃金', 68),
 ('春風', 65),
 ('萬里', 60),
 ('白日', 57),
 ('明月', 52),
 ('青天', 49),
 ('相思', 49),
 ('天地', 47),
 ('青雲', 44)]

In [77]:
def song_author_topK(author,K):
    author_seg = song_df[song_df['author']==author]['dic_total'].tolist()
    if len(author_seg)==0:return []
    author_list = reduce(operator.add, author_seg) 
    result_author = Counter(author_list)
    return  result_author.most_common(K)

In [78]:
song_author_topK('苏轼',10)

[('归去', 44),
 ('何处', 39),
 ('使君', 28),
 ('佳人', 26),
 ('人间', 23),
 ('明月', 22),
 ('江南', 20),
 ('尊前', 20),
 ('千里', 19),
 ('风流', 18)]

In [38]:
list(song_author_topK('苏轼',10))[0][0]

'归去'

In [39]:
song_author_topK('苏轼',10)[0]

('归去', 44)

In [40]:
from cnsenti import Sentiment
from zhconv import convert

In [41]:
senti = Sentiment()
test_text= convert('君不見黃河之水天上來，奔流到海不復回。, 君不見高堂明鏡悲白髮，朝如青絲暮成雪。', 'zh-hans')
result = senti.sentiment_count(test_text)
print(result)

Building prefix dict from C:\Users\JUNHONG\history-digital\lyric_dict.txt ...
Loading model from cache C:\Users\JUNHONG\AppData\Local\Temp\jieba.uaf15fae91440152dd276ee8f618272a6.cache
Loading model cost 0.175 seconds.
Prefix dict has been built successfully.


{'words': 26, 'sentences': 2, 'pos': 1, 'neg': 0}


In [42]:
from zhconv import convert

In [43]:
convert('君不見黃河之水天上來，奔流到海不復回。, 君不見高堂明鏡悲白髮，朝如青絲暮成雪。', 'zh-hans')

'君不见黄河之水天上来，奔流到海不复回。, 君不见高堂明镜悲白发，朝如青丝暮成雪。'

In [44]:
def text_sentiment(str):
    senti = Sentiment()
    test_text= convert(str, 'zh-hans')
    result1 = senti.sentiment_count(test_text)
    result2 = senti.sentiment_calculate(test_text)
    return result1,result2

In [45]:
text_sentiment('君不見黃河之水天上來，奔流到海不復回。, 君不見高堂明鏡悲白髮，朝如青絲暮成雪。')

({'words': 26, 'sentences': 2, 'pos': 1, 'neg': 0},
 {'sentences': 2, 'words': 26, 'pos': 12, 'neg': 0})

In [46]:
text_sentiment('雨恨云愁，江南依旧称佳丽。, 水村渔市。, 一缕孤烟细。')

({'words': 19, 'sentences': 3, 'pos': 2, 'neg': 0},
 {'sentences': 3, 'words': 19, 'pos': 7, 'neg': 0})

In [47]:
city = ['长安','成都','扬州','杭州','苏州','金陵','幽州','明州','洛阳','钱塘','益州','临安','汴州','汴梁','锦官城','锦城','楼兰','渭城','玉门关']

In [48]:
reduce(operator.add,libai['paragraphs'].tolist())

['三十六離宮，樓臺與天通。',
 '閣道步行月，美人愁煙空。',
 '恩疎寵不及，桃李傷春風。',
 '淫樂意何極，金輿向回中。',
 '萬乘出黃道，千旗揚彩虹。',
 '前軍細柳北，後騎甘泉東。',
 '豈問渭川老，寧邀襄野童。',
 '秋暮瑤池宴，歸來樂未窮。',
 '去年戰桑乾源，今年戰蔥河道。',
 '洗兵條支海上波，放馬天山雪中草。',
 '萬里長征戰，三軍盡衰老。',
 '匈奴以殺戮爲耕作，古來唯見白骨黃沙田。',
 '秦家築城備胡處，漢家還有烽火然。',
 '烽火然不息，征戰無已時。',
 '野戰格鬬死，敗馬號鳴向天悲。',
 '烏鳶啄人腸，銜飛上挂枯樹枝。',
 '士卒塗草莽，將軍空爾爲。',
 '乃知兵者是凶器，聖人不得已而用之。',
 '君不見黃河之水天上來，奔流到海不復回。',
 '君不見高堂明鏡悲白髮，朝如青絲暮成雪。',
 '人生得意須盡歡，莫使金尊空對月。',
 '天生我材必有用，千金散盡還復來。',
 '烹羊宰牛且爲樂，會須一飲三百杯。',
 '岑夫子，丹丘生，將進酒，杯莫停。',
 '與君歌一曲，請君爲我側耳聽。',
 '鐘鼓饌玉不足貴，但願長醉不復醒。',
 '古來聖賢皆寂寞，惟有飲者留其名。',
 '陳王昔時宴平樂，斗酒十千恣歡謔。',
 '主人何爲言少錢，徑須酤取對君酌。',
 '五花馬，千金裘，呼兒將出換美酒，與爾同銷萬古愁。',
 '君馬黃，我馬白。',
 '馬色雖不同，人心本無隔。',
 '共作遊冶盤，雙行洛陽陌。',
 '長劒既照曜，高冠何赩赫。',
 '各有千金裘，俱爲五侯客。',
 '猛虎落陷穽，壯夫時屈厄。',
 '相知在急難，獨好亦何益。',
 '我思仙人，乃在碧海之東隅。',
 '海寒多天風，白波連山倒蓬壺。',
 '長鯨噴湧不可涉，撫心茫茫淚如珠。',
 '西來青鳥東飛去，願寄一書謝麻姑。',
 '辟邪伎作鼓吹驚，雉子班之奏曲成。',
 '喔咿振迅欲飛鳴，扇錦翼，雄風生。',
 '雙雌同飲啄，趫悍誰能爭。',
 '乍向草中耿介死，不求黃金籠下生。',
 '天地至廣大，何惜遂物情。',
 '善卷讓天子，務光亦逃名。',
 '所貴曠士懷，朗然合太清。',
 '垂楊拂綠水，搖豔東風年。',
 '花明玉關雪，葉暖金窗煙。',
 '美人結長恨，相對心悽然。',
 '攀條折春色，遠寄龍庭前。',
 '明月出天山，蒼茫雲

In [70]:
def tang_city_count(author):
    result = {}
    for c in city:
        count = 0
        author_l = tang_df[tang_df['author']== author]
        if len( author_l)==0:
            return {}
        l = reduce(operator.add, author_l['paragraphs'].tolist())
        for j in l:
            if c in convert(j, 'zh-hans'):
                count = count + 1
        result[c] = count
    return result
            

In [71]:
def song_city_count(author):
    result = {}
    for c in city:
        count = 0
        author_l = song_df[song_df['author']== author]
        if len( author_l)==0:
            return {}
        l = reduce(operator.add, author_l['paragraphs'].tolist())
        for j in l:
            if c in convert(j, 'zh-hans'):
                count = count + 1
        result[c] = count
    return result

In [53]:
tang_city_count('李白')

{'长安': 35,
 '成都': 1,
 '扬州': 6,
 '杭州': 0,
 '苏州': 0,
 '金陵': 30,
 '幽州': 7,
 '明州': 0,
 '洛阳': 26,
 '钱塘': 0,
 '益州': 0,
 '临安': 1,
 '汴州': 0,
 '汴梁': 0,
 '锦官城': 0,
 '锦城': 6,
 '楼兰': 5,
 '渭城': 1,
 '玉门关': 2}

In [51]:
tang_female = ['薛涛','刘采春','鱼玄机','李冶']

In [62]:
tang_author_topK('薛濤',10)

[('不得', 13),
 ('芙蓉', 6),
 ('花落', 4),
 ('同心', 4),
 ('人間', 4),
 ('夕陽', 4),
 ('四五', 4),
 ('春風', 3),
 ('今日', 3),
 ('萬里', 3)]

In [63]:
for i in tang_female:
    print(i,tang_city_count(convert(i, 'zh-hant')))

薛涛 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 1, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 1, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
刘采春 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
鱼玄机 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
李冶 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}


In [64]:
for i in tang_female:
    print(i,tang_author_topK(convert(i, 'zh-hant'),10))

薛涛 [('不得', 13), ('芙蓉', 6), ('花落', 4), ('同心', 4), ('人間', 4), ('夕陽', 4), ('四五', 4), ('春風', 3), ('今日', 3), ('萬里', 3)]
刘采春 [('桐廬', 2), ('昨日', 2), ('不喜', 1), ('秦淮', 1), ('生憎', 1), ('江上', 1), ('載兒', 1), ('夫壻', 1), ('經歲', 1), ('又經', 1)]
鱼玄机 [('何處', 6), ('鸚鵡', 4), ('多情', 4), ('何事', 3), ('滿庭', 3), ('鴛鴦', 3), ('朱弦', 3), ('楊柳', 3), ('客舟', 2), ('風雨', 2)]
李冶 [('相思', 6), ('相逢', 3), ('一時', 3), ('寂寞', 2), ('別後', 2), ('巫山', 2), ('流泉', 2), ('一彈', 2), ('海水', 2), ('暮雨', 2)]


In [66]:
song_female = ['李清照','张玉娘','吴淑姬','朱淑真']

In [72]:
for i in song_female:
    print(i,song_city_count(convert(i, 'zh-hant')))

李清照 {'长安': 1, '成都': 0, '扬州': 1, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}
张玉娘 {}
吴淑姬 {}
朱淑真 {'长安': 0, '成都': 0, '扬州': 0, '杭州': 0, '苏州': 0, '金陵': 0, '幽州': 0, '明州': 0, '洛阳': 0, '钱塘': 0, '益州': 0, '临安': 0, '汴州': 0, '汴梁': 0, '锦官城': 0, '锦城': 0, '楼兰': 0, '渭城': 0, '玉门关': 0}


In [79]:
for i in song_female:
    print(i,song_author_topK(convert(i, 'zh-hant'),10))

李清照 [('多少', 6), ('萧萧', 6), ('憔悴', 6), ('瑞脑', 6), ('情怀', 5), ('香消', 5), ('梅花', 5), ('旧时', 5), ('庭院', 5), ('如今', 5)]
张玉娘 []
吴淑姬 []
朱淑真 [('寂寞', 6), ('何处', 6), ('东风', 5), ('阑干', 5), ('芳草', 5), ('不忍', 4), ('梨花', 4), ('起来', 4), ('一枝', 4), ('恼人', 3)]
